<a href="https://colab.research.google.com/github/bartek717/projectx/blob/main/projectxEval%2BHallucinationMetric.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install bert-score
!pip install nltk
!pip install pyrouge
!pip install sentencepiece
!pip install datasets
!pip install transformers[torch]
!pip install accelerate -U
!pip install evaluate
!pip install rouge_score
!pip install openpyxl
!pip install torch
!pip install summac

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 458.9 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyrouge: filename=pyrouge-0.1.3-py3-none-any.whl size=191605 sha256=ef8b1f8692f91da9a8a616c6c20bda9b08bb27e7da529e87a9d6da176d36a319
  Stored in directory: /root/.cache/pip/wheels/9a/67/12/c5dd8ef8b4152bb8789eafd2a74a734e2dc7bb9eae02b768e7
Successfully built pyrouge
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 19.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.7/270.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.0 MB/s eta 0:00:00
  Preparing metadata (

In [ ]:
# evaluation of the models

In [ ]:
!pip install --upgrade transformers huggingface_hub

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 330.3/330.3 kB 31.3 MB/s eta 0:00:00
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.17.0
    Uninstalling huggingface-hub-0.17.0:
      Successfully uninstalled huggingface-hub-0.17.0
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
summac 0.0.4 requires huggingface-hub<=0.17.0, but you have huggingface-hub 0.20.2 which is incompatible.


In [ ]:
from nltk.translate.bleu_score import corpus_bleu
from bert_score import score
from pyrouge import Rouge155
from transformers import T5Tokenizer, T5Model
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainingArguments, Seq2SeqTrainer
from datasets import load_dataset, Dataset
import torch
import pandas as pd
from google.colab import files
import time
import os
import numpy as np
import evaluate
from nltk.tokenize import sent_tokenize
import nltk
import random
from summac.model_summac import SummaCZS, SummaCConv

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# import models

In [ ]:
!git clone https://github.com/vgupta123/sumpubmed.git
%cd sumpubmed/

Cloning into 'sumpubmed'...
remote: Enumerating objects: 130808, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 130808 (delta 22), reused 17 (delta 6), pack-reused 130764
Receiving objects: 100% (130808/130808), 346.82 MiB | 17.71 MiB/s, done.
Resolving deltas: 100% (38368/38368), done.
Updating files: 100% (130765/130765), done.
/content/sumpubmed


In [ ]:
def process_data(text_dir, summary_dir):
    texts = []
    summaries = []
    for filename in os.listdir(text_dir):
        if filename.endswith(".txt"):
            with open(os.path.join(text_dir, filename), 'r', encoding='utf-8') as file:
                texts.append(file.read())

            summary_file = "abst_"+(filename.split("_")[-1])
            with open(os.path.join(summary_dir, summary_file), 'r', encoding='utf-8') as file:
                summaries.append(file.read())

    return texts, summaries

texts_full, summaries_full = process_data('line_text/', 'shorter_abstract/')

# TEMP, SHOULD BE CHANGED TO SOMETHING STANDARDIZED
texts = texts_full[int(0.999*len(texts_full)):]
reference_summaries = summaries_full[int(0.999*len(summaries_full)):]

In [ ]:
def clean_text(text):
    # Replace <cit> and <dig> with an empty string
    cleaned_text = text.replace("<cit>", "").replace("<dig>", "")
    return cleaned_text

texts = [clean_text(text) for text in texts]
reference_summaries = [clean_text(summary) for summary in reference_summaries]


In [ ]:
print(len(texts))
def generateText(tokenizer, model):
    generation_parameters = {
        "max_new_tokens": 256,
        "num_beams": 5,
        "early_stopping": True,
        "no_repeat_ngram_size": 3,
        "temperature": 0.3
    }

    model_summaries = []
    for text in texts:
        inputs = tokenizer.encode(text, return_tensors="pt", max_length=1024, truncation=True).to(device)  # Move inputs to the same device
        summary_ids = model.generate(inputs, **generation_parameters)
        generated_summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        model_summaries.append(generated_summary)

    return model_summaries

33


In [ ]:
# evaluate models

In [ ]:
rouge_score = evaluate.load("rouge")

def compute_rouge_score(generated, reference):
    """
    Calculates and returns the ROUGE score using the evaluate library.
    """
    generated_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in generated]
    reference_with_newlines = ["\n".join(sent_tokenize(s.strip())) for s in reference]

    rouge_scores = rouge_score.compute(
        predictions=generated_with_newlines,
        references=reference_with_newlines,
        use_stemmer=True
    )

    return rouge_scores


In [ ]:
def bleu_mean(reference_summaries, model_summaries):
    """
    Calculates and returns the mean BLEU score.
    """
    bleu_scores = corpus_bleu(reference_summaries, model_summaries)
    return bleu_scores

In [ ]:
def bert_score_mean(model_summaries, reference_summaries, lang="en"):
    """
    Calculates and returns the mean BertScore.
    """
    P, R, F1 = score(model_summaries, reference_summaries, lang=lang)
    mean_f1_score = F1.mean().item()

    return mean_f1_score

In [ ]:
# Preparing for hallucination evaluation.
model_conv = SummaCConv(models=["vitc"], bins='percentile', granularity="sentence", nli_labels="e", device="cuda", start_file="default", agg="mean")

<All keys matched successfully>


In [ ]:
model_zs = SummaCZS(granularity="sentence", model_name="vitc", device="cuda")

In [ ]:
def model_conv_hallucination_score(model_summaries, reference_summaries):
    """
    Calculates the average hallucination score in summarized text.
    """
    score_conv1 = 0
    for i in range(len(model_summaries)):
        score = model_conv.score([model_summaries[i]], [reference_summaries[i]])
        score_conv1 += score["scores"][0]
        print("Index:", i)
        print("Score:", score["scores"][0])

    mean_score_conv1 = 0
    if len(model_summaries) > 0:
        mean_score_conv1 = score_conv1 / len(model_summaries)

    return mean_score_conv1

In [ ]:
def model_zs_hallucination_score(model_summaries, reference_summaries):
    """
    Calculates the average hallucination score in summarized text.
    """
    score_zs1 = 0
    for i in range(len(model_summaries)):
        score = model_zs.score([model_summaries[i]], [reference_summaries[i]])
        score_zs1 += score["scores"][0]
        print("Index:", i)
        print("Score:", score["scores"][0])

    mean_score_zs1 = 0
    if len(model_summaries) > 0:
        mean_score_zs1 = score_zs1 / len(model_summaries)

    return mean_score_zs1

Quick test to see if hallucination function runs properly

In [ ]:
model_conv_hallucination_score(reference_summaries, texts)

Index: 0
Score: 0.22198162972927094
Index: 1
Score: 0.23150542378425598
Index: 2
Score: 0.23831652104854584
Index: 3
Score: 0.2266944944858551
Index: 4
Score: 0.2573741674423218
Index: 5
Score: 0.21115198731422424
Index: 6
Score: 0.23443330824375153
Index: 7
Score: 0.2277088463306427
Index: 8
Score: 0.210519939661026
Index: 9
Score: 0.25779497623443604
Index: 10
Score: 0.23967288434505463
Index: 11
Score: 0.214077427983284
Index: 12
Score: 0.21734189987182617
Index: 13
Score: 0.2124471664428711
Index: 14
Score: 0.2324167937040329
Index: 15
Score: 0.22734995186328888
Index: 16
Score: 0.2162012904882431
Index: 17
Score: 0.2803749442100525
Index: 18
Score: 0.21170024573802948
Index: 19
Score: 0.2380165010690689
Index: 20
Score: 0.2210833728313446
Index: 21
Score: 0.2094363421201706
Index: 22
Score: 0.21602913737297058
Index: 23
Score: 0.2180432826280594
Index: 24
Score: 0.22271424531936646
Index: 25
Score: 0.20560751855373383
Index: 26
Score: 0.22735773026943207
Index: 27
Score: 0.2131873

0.22788782282309097

In [ ]:
model_zs_hallucination_score(reference_summaries, texts)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/217 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.09k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/760k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/156 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/235M [00:00<?, ?B/s]

Index: 0
Score: -0.10542897458346386
Index: 1
Score: -0.16817959717341832
Index: 2
Score: -0.13923487704024357
Index: 3
Score: -0.16853025743177721
Index: 4
Score: -0.11160591217686402
Index: 5
Score: -0.1319667328235715
Index: 6
Score: -0.12388934446184822
Index: 7
Score: -0.0881814667672822
Index: 8
Score: -0.07858820208187761
Index: 9
Score: -0.09908621918921377
Index: 10
Score: -0.15755586594061144
Index: 11
Score: -0.14611901432634836
Index: 12
Score: -0.11475511996642403
Index: 13
Score: -0.13351694175175258
Index: 14
Score: -0.08147743675443861
Index: 15
Score: -0.165472429852153
Index: 16
Score: -0.11915263597545706
Index: 17
Score: -0.06978516008868907
Index: 18
Score: -0.12688348676774885
Index: 19
Score: -0.07827359538967327
Index: 20
Score: -0.06801859283447266
Index: 21
Score: -0.08010216057300568
Index: 22
Score: -0.15793595040679737
Index: 23
Score: -0.11262920379638672
Index: 24
Score: -0.20840172434962073
Index: 25
Score: -0.08107664625523454
Index: 26
Score: -0.163093

-0.12289443381060246

In [ ]:
def mainFunc(model, tokenizer, texts, reference_summaries):
    model_summaries = generateText(tokenizer, model)
    print(model_summaries[0])
    rouge_scores = compute_rouge_score(model_summaries, reference_summaries)
    mean_bleu_score = bleu_mean(reference_summaries, model_summaries)
    mean_bert_score = bert_score_mean(model_summaries, reference_summaries, lang="en")

    rouge1_f = rouge_scores["rouge1"]
    rouge2_f = rouge_scores["rouge2"]
    rougeL_f = rouge_scores["rougeL"]

    mean_summac_conv = model_conv_hallucination_score(model_summaries, reference_summaries)

    print(f"ROUGE-1 F1 Score: {rouge1_f:.4f}")
    print(f"ROUGE-2 F1 Score: {rouge2_f:.4f}")
    print(f"ROUGE-L F1 Score: {rougeL_f:.4f}")
    print(f"Mean BLEU Score: {mean_bleu_score:.4f}")
    print(f"Mean BertScore: {mean_bert_score:.4f}")
    print(f"Mean SummaC-Conv Score: {mean_summac_conv:.4f}")

    return rouge1_f, rouge2_f, rougeL_f, mean_bleu_score, mean_bert_score, mean_summac_conv

In [ ]:
model_output_dir_full = '/content/drive/My Drive/t5fine'
tokenizer = AutoTokenizer.from_pretrained("google/flan-t5-small")
# look into this
tokenizer.model_max_length=4096
percents = [0.70, 0.50, 0.30, 0.10, 0.05, 0.03, 0.01]
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# model_cg_70 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_70)
# model_cg_50 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_50)
# model_cg_30 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_30)
# model_cg_10 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_10)
# model_cg_5 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_5)
# model_cg_3 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_3)
# model_cg_1 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_cg_1)

# model_al_70 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_70)
# model_al_50 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_50)
# model_al_30 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_30)
# model_al_10 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_10)
# model_al_5 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_5)
# model_al_3 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_3)
# model_al_1 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_al_1)

# model_alcg_70 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_70)
# model_alcg_50 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_50)
# model_alcg_30 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_30)
# model_alcg_10 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_10)
# model_alcg_5 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_5)
# model_alcg_3 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_3)
# model_alcg_1 = AutoModelForSeq2SeqLM.from_pretrained(model_output_dir_alcg_1)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

In [ ]:
output_dirs = [model_output_dir_full]
for percent in percents:
    model_output_dir = '/content/drive/My Drive/t5finekcenter_' + str(percent)[-2:]
    output_dirs.append(model_output_dir)


for percent in percents:
    random_output_dir = '/content/drive/My Drive/t5finerandom_' + str(percent)[-2:]
    output_dirs.append(random_output_dir)

print(output_dirs)

['/content/drive/My Drive/t5fine', '/content/drive/My Drive/t5finekcenter_.7', '/content/drive/My Drive/t5finekcenter_.5', '/content/drive/My Drive/t5finekcenter_.3', '/content/drive/My Drive/t5finekcenter_.1', '/content/drive/My Drive/t5finekcenter_05', '/content/drive/My Drive/t5finekcenter_03', '/content/drive/My Drive/t5finekcenter_01', '/content/drive/My Drive/t5finerandom_.7', '/content/drive/My Drive/t5finerandom_.5', '/content/drive/My Drive/t5finerandom_.3', '/content/drive/My Drive/t5finerandom_.1', '/content/drive/My Drive/t5finerandom_05', '/content/drive/My Drive/t5finerandom_03', '/content/drive/My Drive/t5finerandom_01']


In [ ]:
def append_to_excel(file_path, data, sheet_name='Sheet1'):
    try:
        df_existing = pd.read_excel(file_path, sheet_name=sheet_name)
    except FileNotFoundError:
        df_existing = pd.DataFrame()
    df_new = pd.DataFrame(data)
    df_final = df_existing.append(df_new, ignore_index=True)
    df_final.to_excel(file_path, sheet_name=sheet_name, index=False)

In [ ]:
print(output_dirs)
for dir in output_dirs:
    model = AutoModelForSeq2SeqLM.from_pretrained(dir)

    model = model.to(device)
    rouge1_f, rouge2_f, rougeL_f, mean_bleu_score, mean_bert_score, summac_conv_score = mainFunc(model, tokenizer, texts, reference_summaries)
    data_to_append = {
        'Model Directory': [dir],
        'ROUGE1': [str(rouge1_f)],
        'ROUGE2': [str(rouge2_f)],
        'ROUGEL': [str(rougeL_f)],
        'Mean BLEU Score': [str(mean_bleu_score)],
        'Mean BERT Score': [str(mean_bert_score)],
        'Mean SummaC-Conv Score': [str(summac_conv_score)]
    }
    append_to_excel('results.xlsx', data_to_append)


['/content/drive/My Drive/t5fine', '/content/drive/My Drive/t5finekcenter_.7', '/content/drive/My Drive/t5finekcenter_.5', '/content/drive/My Drive/t5finekcenter_.3', '/content/drive/My Drive/t5finekcenter_.1', '/content/drive/My Drive/t5finekcenter_05', '/content/drive/My Drive/t5finekcenter_03', '/content/drive/My Drive/t5finekcenter_01', '/content/drive/My Drive/t5finerandom_.7', '/content/drive/My Drive/t5finerandom_.5', '/content/drive/My Drive/t5finerandom_.3', '/content/drive/My Drive/t5finerandom_.1', '/content/drive/My Drive/t5finerandom_05', '/content/drive/My Drive/t5finerandom_03', '/content/drive/My Drive/t5finerandom_01']


HFValidationError: ignored

In [ ]:
# do t-tests/anova for statistical significance